In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.stattools import acf , pacf
from stats models.tsa.statespace.sarimax import SARIMAX
register_matplotlib_converters()
from time import time

# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error

In [ ]:
sharston = pd.read_csv('./sharston_combined.csv',header=0,index_col=0)
picadilly = pd.read_csv('./MCC_combined.csv',header=0,index_col=0)

In [ ]:
sharston.index = pd.DatetimeIndex(sharston.index,freq='h')
picadilly.index = pd.DatetimeIndex(picadilly.index,freq='h')

In [ ]:
train_start , train_end = '2019-01-01' , '2020-01-01'
test_start, test_end = '2020-03-23' , '2020-04-23'

In [ ]:
train_dates = picadilly[(picadilly.index >= train_start) & (picadilly.index < train_end ) ].index.to_numpy()
test_dates = picadilly[(picadilly.index >= test_start) & (picadilly.index <= test_end ) ].index.to_numpy()

train_shar = sharston[(sharston.index >= train_start ) & (sharston.index < train_end ) ][['NO2']]
test_shar = sharston[(sharston.index >= test_start ) & (sharston.index <= test_end ) ][['NO2']]

train_picadilly = picadilly[(picadilly.index >= train_start) & (picadilly.index < train_end ) ][['NO2']]
test_picadilly = picadilly[(picadilly.index >= test_start) & (picadilly.index <= test_end ) ][['NO2']]

plt.figure(figsize=(10,4))
plt.plot(train_shar)

difference = 3
num_lags = 24
lags = np.arange(0,num_lags)

plt.figure(figsize=(10,4))
plt.plot(train_shar.diff(difference)[difference:])
plt.title(f' I = {difference} Data')
plt.show()

acf_vals = acf(train_shar.diff(difference)[difference:], nlags=num_lags)

plt.figure(figsize=(10,4))
plt.bar(lags,acf_vals[:num_lags])
plt.title (f'ACF: difference {difference}, nlags{num_lags} ')
plt.show()

pacf_vals = pacf(train_shar.diff(difference)[difference:], nlags=num_lags)

plt.figure(figsize=(10,4))
plt.bar(lags,pacf_vals[:num_lags])
plt.title (f'PACF: difference {difference}, nlags{num_lags} ')
plt.show()



In [ ]:
order = (1,1,1)
seasonal_order = (1,1,1,24)
model = SARIMAX(train_shar[:24*7],order=order,seasonal_order=seasonal_order)
model_fit = model.fit()



In [ ]:
last_date = model_fit.forecast(1).index[0]
last_date

In [ ]:
df = train_shar[:5]
df

In [ ]:
df = df.append(pd.DataFrame(index=[last_date]))
df.loc[last_date]['NO2'] = 2
df

In [ ]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[24]):
    models = list()
    # define config lists
    p_params = [1, 2]
    d_params = [0, 1]
    q_params = [1, 2]
    t_params = ['t']
    P_params = [1, 2]
    D_params = [1]
    Q_params = [1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [ ]:
configs = sarima_configs()

In [ ]:
results = []
counter = 0
days = 7
forecast = 24
try:
    # never show warnings when grid searching, too noisy
    with catch_warnings():
        filterwarnings("ignore")
        for cfg in range (len(configs[:1])):
            print (configs[cfg])
            for j in range (0, len(train_shar), (24*days))[::10]:
                print (f'Block Startdate: {train_shar.iloc[j].name}')
                counter +=1
                errors = []
                for i in range ( forecast ):
                    start = time()
                    order , seasonal_order , trend = configs[cfg]
                    train = train_shar.iloc[ j : j +(24*days)+i]
                    model = SARIMAX(train, order=order,seasonal_order=seasonal_order, trend = trend )
                    model_fit = model.fit()
                    
                    
                    predicted = model_fit.forecast(1)[0]
                    date = model_fit.forecast(1).index[0]
                    
                    observed = train_shar.loc[date][0]
                    
                    error =  ((predicted - observed)**2)**.5 
                    errors.append(error)
                    results.append([counter, date, predicted, observed, error, configs[cfg]])
                
                    print ( f'Block:{counter}, Date:{date}, Predicted:{predicted} ,Observed:{observed}, Error:{error} ' )
                RMSE = np.mean( errors )
                print (f'Block: {counter}, RMSE:{ RMSE }')
                end = time()
                
                print (end-start)
#                     print ( f'Date:{train_shar.iloc[j +(24*days)+i+1].name}, Predicted:{model_fit.forecast(1)[0]} ,Observed:{train_shar.iloc[j +(24*days)+i+1][0]}' )

        
except:
    error = None

In [ ]:
#rolling version

results = []
counter = 0
days = 7
forecast = 3
try:
    # never show warnings when grid searching, too noisy
    with catch_warnings():
        filterwarnings("ignore")
        
        for cfg in range (len(configs[:1])):
            print (configs[cfg])
            for j in range (0, len(train_shar), (24*days))[::10]:
                counter +=1
                for t in range (0,24,forecast):
                    print (f'Block Startdate: {train_shar.iloc[j].name}')
                    errors = []
                    train = train_shar.iloc[ j+t : j+t +(24*days)]
                    for i in range ( forecast ):

                        start = time()
                        order , seasonal_order , trend = configs[cfg]
                        model = SARIMAX(train, order=order,seasonal_order=seasonal_order, trend = trend )
                        model_fit = model.fit()


                        predicted = model_fit.forecast(1)[0]
                        date = model_fit.forecast(1).index[0]

                        observed = train_shar.loc[date][0]

                        error =  ((predicted - observed)**2)**.5 
                        errors.append(error)
                        results.append([counter, date, predicted, observed, error, configs[cfg]])

                        train = train.append(pd.DataFrame(index=[date]))

                        train.loc[date]['NO2'] = predicted
                        train.index = pd.DatetimeIndex(train.index,freq='h')

                        print ( f'Block:{counter}, Date:{date}, Predicted:{predicted} ,Observed:{observed}, Error:{error} ' )
                    RMSE = np.mean( errors )
                    print (f'Block: {counter}, RMSE:{ RMSE }')
                    end = time()

                    print (end-start)
        #                     print ( f'Date:{train_shar.iloc[j +(24*days)+i+1].name}, Predicted:{model_fit.forecast(1)[0]} ,Observed:{train_shar.iloc[j +(24*days)+i+1][0]}' )

        
except:
    error = None

In [ ]:
test_shar